Factor data download -- Financial reporting factor

Data source: TuShare

Period: 12/31/1999 - 12/31/2021

Sample frequency: monthly, quarterly
    
Data: fina_indicator & float type data in “财报指标.csv”

In [2]:
pip install tushare

     |████████████████████████████████| 133kB 267kB/s eta 0:00:01
     |████████████████████████████████| 61kB 248kB/s eta 0:00:01
     |████████████████████████████████| 81kB 695kB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-cp37-none-any.whl size=1273 sha256=9e55e0394bb79b6f64d497fe4ee975f4993b027ca38fca72610ec0b3bcfc4afc
  Stored in directory: /Users/cr/Library/Caches/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
import pandas as pd
import tushare as ts
from tqdm import tqdm
pro = ts.pro_api("eb13b3bfd2bd07fd9eb40234f19941c73f230e1e98cc212b8cd407c7")

data_l = pro.stock_basic(exchange='', list_status='L', 
                         fields='ts_code,list_status,name,area,industry,list_date,delist_date,is_hs')
data_d = pro.stock_basic(exchange='', list_status='D', 
                         fields='ts_code,list_status,name,area,industry,list_date,delist_date,is_hs')
data_p = pro.stock_basic(exchange='', list_status='P', 
                         fields='ts_code,list_status,name,area,industry,list_date,delist_date,is_hs')

basic_data = pd.concat([data_l,data_d,data_p])
codelist = list(basic_data['ts_code'])

# 读取财报因子

In [2]:
file = '/Users/cr/Downloads/undergraduate time/大四上/FIN 4998/paper/code/data & data code/'
fdf = pd.read_csv(file+'财报指标.csv',sep=',')
type_list = ['fina_indicator']

In [ ]:
data = pd.DataFrame()
for sheet_type in type_list:
    f = list(fdf.query(f'表格类型=="{sheet_type}"')['名称'])
    for i in ['report_type','comp_type','f_ann_date']:
        try:
            f.remove(i)
        except:
            pass
    f = ','.join(f)
    fdata = []
    
    for c in tqdm(codelist):
        temp = pro.query(sheet_type, ts_code=c, fields=f,
                             start_date='19991231', end_date='20211231')

        temp = temp.set_index(['ts_code','ann_date','end_date'])
  
        '''
        删除重复索引，且update_flag为0的，根据tushare描述，为1的是修改过的。就用修改过的吧。
        counter是计数，dict是创建了个对索引的计数字典。d.items()是字典中所有键值对
        '''

        from collections import Counter
        d = dict(Counter(temp.index))
        t1 = temp.loc[[key for key,value in d.items() if value>1],:]
        t1 = t1.fillna(method='ffill')
        t1 = t1.fillna(method='bfill')
        t1 = t1.loc[~t1.index.duplicated(keep='first')]
        t0 = temp.loc[[key for key,value in d.items() if value==1],:]
        temp = pd.concat([t0,t1],axis=0)
        
        
        temp = temp.reset_index(['ann_date'])
        temp = temp.drop(labels=['update_flag'],axis=1)  #删除4个无用的列
        d = dict(Counter(temp.index))
        t1 = temp.loc[[key for key,value in d.items() if value>1],:]
        t1 = t1.sort_values(by='ann_date')  #按时间先后顺序排
        t1 = t1.fillna(method='ffill')  #前向填充，即如果同一个财报，后面的数据缺失，用前面的数据填充
        try:
            t0 = temp.loc[[key for key,value in d.items() if value==1],:]
            temp = pd.concat([t0,t1],axis=0)
        except:
            temp = t1.copy()
        temp = temp.reset_index(['ts_code','end_date'])
        
        
        col = temp.columns
        temp = temp.values.tolist()
        fdata.extend(temp)
        time.sleep(0.2)  
        #即每秒5个，按照每分钟200次上限。要低于3.33it/s，sleep上限最大为大约0.3,这个目测0.15就稳定再3it/s以下了。
    fdata = pd.DataFrame(fdata,columns=col)
    fdata = fdata.set_index(['ts_code','ann_date','end_date'])

    data = pd.concat([data,fdata],axis=1)

In [7]:
import pickle 
pickle.dump(data,open(file+'fina_data','wb'))